In [4]:
### THALAMUS INSTRUCTIONS
### create dscalar files with only thalamus and only cerebellum with this: /Applications/workbench/bin_macosx64/wb_command -cifti-label-to-roi /Users/xavierguell/Desktop/HCP_parcels/Gordon_333_Parcel+FS_Anatomical_Parcel.dlabel.nii thalamus_left.dscalar.nii -key 347
### atlas file is what they sent me through the HCP mailing list

## transform these dscalar files to nifti volume files with this:
## /Applications/workbench/bin_macosx64/wb_command -cifti-separate thalamus_left.dscalar.nii COLUMN -volume-all thalamus_left.nii

## restrict dconn file to each of these structures (possibly also adding thalamus left plus right, or left thalamus plus right cerebellum) by combining the .nii files with fslmaths, with this:
## /Applications/workbench/bin_macosx64/wb_command -cifti-restrict-dense-map /Users/xavierguell/Desktop/HCP_S1200_1003_rfMRI_MSMAll_groupPCA_d4500ROW_zcorr.dconn.nii COLUMN dconn_thalamus_left_previousstep.dconn.nii -vol-roi thalamus_left.nii
## /Applications/workbench/bin_macosx64/wb_command -cifti-restrict-dense-map dconn_thalamus_left_previousstep.dconn.nii ROW dconn_thalamus_left.dconn.nii -vol-roi thalamus_left.nii

## restrict a random timeseries from HCP to my structure only with -cifti-restrict-dense-map, and then merge it to have the number of maps as indicated in emb.shape (for bilateral thalamus it is 14):
## /Applications/workbench/bin_macosx64/wb_command -cifti-merge mergedfile14_thalamus_bilateral.dtseries.nii -cifti cope1_thalamus_bilateral.dtseries.nii -cifti cope1_thalamus_bilateral.dtseries.nii -cifti cope1_thalamus_bilateral.dtseries.nii -cifti cope1_thalamus_bilateral.dtseries.nii -cifti cope1_thalamus_bilateral.dtseries.nii -cifti cope1_thalamus_bilateral.dtseries.nii -cifti cope1_thalamus_bilateral.dtseries.nii -cifti cope1_thalamus_bilateral.dtseries.nii -cifti cope1_thalamus_bilateral.dtseries.nii -cifti cope1_thalamus_bilateral.dtseries.nii -cifti cope1_thalamus_bilateral.dtseries.nii -cifti cope1_thalamus_bilateral.dtseries.nii -cifti cope1_thalamus_bilateral.dtseries.nii -cifti cope1_thalamus_bilateral.dtseries.nii
## /Applications/workbench/bin_macosx64/wb_command -cifti-change-mapping mergedfile14_thalamus_bilateral.dtseries.nii ROW mergedfile14_thalamus_bilateral.dscalar.nii -scalar -name-file dscalar_names.txt


In [3]:
import numpy as np
import nibabel as nib
from sklearn.metrics import pairwise_distances
dcon = nib.load('dconn_thalamus_bilateral.dconn.nii').get_data()

In [4]:
dcon.shape

(2536, 2536)

In [14]:
#(0, 0, 'CIFTI_STRUCTURE_CORTEX_LEFT')
#(1, 29696, 'CIFTI_STRUCTURE_CORTEX_RIGHT')
#(2, 59412, 'CIFTI_STRUCTURE_ACCUMBENS_LEFT')
#(3, 59547, 'CIFTI_STRUCTURE_ACCUMBENS_RIGHT')
#(4, 59687, 'CIFTI_STRUCTURE_AMYGDALA_LEFT')
#(5, 60002, 'CIFTI_STRUCTURE_AMYGDALA_RIGHT')
#(6, 60334, 'CIFTI_STRUCTURE_BRAIN_STEM')
#(7, 63806, 'CIFTI_STRUCTURE_CAUDATE_LEFT')
#(8, 64534, 'CIFTI_STRUCTURE_CAUDATE_RIGHT')
#(9, 65289, 'CIFTI_STRUCTURE_CEREBELLUM_LEFT')
#(10, 73998, 'CIFTI_STRUCTURE_CEREBELLUM_RIGHT')
#(11, 83142, 'CIFTI_STRUCTURE_DIENCEPHALON_VENTRAL_LEFT')
#(12, 83848, 'CIFTI_STRUCTURE_DIENCEPHALON_VENTRAL_RIGHT')
#(13, 84560, 'CIFTI_STRUCTURE_HIPPOCAMPUS_LEFT')
#(14, 85324, 'CIFTI_STRUCTURE_HIPPOCAMPUS_RIGHT')
#(15, 86119, 'CIFTI_STRUCTURE_PALLIDUM_LEFT')
#(16, 86416, 'CIFTI_STRUCTURE_PALLIDUM_RIGHT')
#(17, 86676, 'CIFTI_STRUCTURE_PUTAMEN_LEFT')
#(18, 87736, 'CIFTI_STRUCTURE_PUTAMEN_RIGHT')
#(19, 88746, 'CIFTI_STRUCTURE_THALAMUS_LEFT')
#(20, 90034, 'CIFTI_STRUCTURE_THALAMUS_RIGHT')

In [5]:
# Generate .dscalar file with the 8 gradients
import numpy as np
import nibabel as nib
from sklearn.metrics import pairwise_distances
dcon = np.tanh(dcon) ## this step takes a while
N = dcon.shape[0]
perc = np.array([np.percentile(x, 90) for x in dcon])
for i in range(dcon.shape[0]):
    print "Row %d" % i
    dcon[i, dcon[i,:] < perc[i]] = 0
dcon[dcon < 0] = 0
aff = 1 - pairwise_distances(dcon, metric = 'cosine')
np.save('cosine_affinity_thalamus_bilateral.npy', aff)
aff = np.load('cosine_affinity_thalamus_bilateral.npy')
import sys
sys.path.append("./mapalign")
from mapalign import embed
emb, res = embed.compute_diffusion_map(aff, alpha = 0.5)
np.save('embedding_dense_thalamus_bilateral_emb.npy', emb)
np.save('embedding_dense_thalamus_bilateral_res.npy', res)

res = np.load('embedding_dense_thalamus_bilateral_res.npy').item()
a = [res['vectors'][:,i]/ res['vectors'][:,0] for i in range(9)]
emb = np.array(a)[1:,:].T
len(emb)
res = nib.load('hcp.tmp.lh.dscalar.nii').get_data()
cortL = np.squeeze(np.array(np.where(res != 0) [0], dtype=np.int32))
res = nib.load('hcp.tmp.rh.dscalar.nii').get_data()
cortR = np.squeeze(np.array(np.where(res != 0) [0], dtype=np.int32))
corLen = len(cortL) + len(cortR)
del res
emb = np.load('embedding_dense_thalamus_bilateral_emb.npy')
emb.shape
######


Row 0
Row 1
Row 2
Row 3
Row 4
Row 5
Row 6
Row 7
Row 8
Row 9
Row 10
Row 11
Row 12
Row 13
Row 14
Row 15
Row 16
Row 17
Row 18
Row 19
Row 20
Row 21
Row 22
Row 23
Row 24
Row 25
Row 26
Row 27
Row 28
Row 29
Row 30
Row 31
Row 32
Row 33
Row 34
Row 35
Row 36
Row 37
Row 38
Row 39
Row 40
Row 41
Row 42
Row 43
Row 44
Row 45
Row 46
Row 47
Row 48
Row 49
Row 50
Row 51
Row 52
Row 53
Row 54
Row 55
Row 56
Row 57
Row 58
Row 59
Row 60
Row 61
Row 62
Row 63
Row 64
Row 65
Row 66
Row 67
Row 68
Row 69
Row 70
Row 71
Row 72
Row 73
Row 74
Row 75
Row 76
Row 77
Row 78
Row 79
Row 80
Row 81
Row 82
Row 83
Row 84
Row 85
Row 86
Row 87
Row 88
Row 89
Row 90
Row 91
Row 92
Row 93
Row 94
Row 95
Row 96
Row 97
Row 98
Row 99
Row 100
Row 101
Row 102
Row 103
Row 104
Row 105
Row 106
Row 107
Row 108
Row 109
Row 110
Row 111
Row 112
Row 113
Row 114
Row 115
Row 116
Row 117
Row 118
Row 119
Row 120
Row 121
Row 122
Row 123
Row 124
Row 125
Row 126
Row 127
Row 128
Row 129
Row 130
Row 131
Row 132
Row 133
Row 134
Row 135
Row 136
Row 137
Row 13

pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


(2536, 14)

In [7]:
emb, res = embed.compute_diffusion_map(aff, alpha = 0.5)

In [12]:
tmp = nib.load('mergedfile14_thalamus_bilateral.dscalar.nii')
tmp_cifti = nib.cifti2.load('mergedfile14_thalamus_bilateral.dscalar.nii')
data = tmp_cifti.get_data() * 0
mim = tmp.header.matrix[1]
for idx, bm in enumerate(mim.brain_models):
    print ((idx, bm.index_offset, bm.brain_structure))
img = nib.cifti2.Cifti2Image(emb.T, nib.cifti2.Cifti2Header(tmp.header.matrix))
img.to_filename('result_thalamus_bilateral.dscalar.nii')

(0, 0, 'CIFTI_STRUCTURE_THALAMUS_LEFT')
(1, 1288, 'CIFTI_STRUCTURE_THALAMUS_RIGHT')
